In [ ]:
import pandas as pd
from pybedtools import BedTool

**INDEX**

**Data processing**

Human high-frequency changes within Deserts

Human high-frequency changes not in Deserts

**1. Deserts vs NO Deserts (Whole genome)** 
    
    Difference mean frequency
    Difference percentage fixed alleles
    
**2. Mean frequency Deserts vs NO Deserts (per chromosome)** 

    Difference mean frequency
    Difference percentage fixed alleles
    
**3. Genes associated to fixed alleles within Deserts**


### **Data processing** 

#### Intersection hHF with Deserts 

In [ ]:
akey = BedTool('/home/juanandres/Desktop/TFM_raul/tmp_section1/2020_akeydeserts_coords.bed')
print(len(akey))
#hf = BedTool('/home/juanandres/Desktop/TFM_raul/tmp_section1/2020_Nahigh_freq.bed')
#print(len(hf))
pey = BedTool('/home/juanandres/Desktop/TFM_raul/tmp_section1/2020_pey_coords.bed')
print(len(pey))

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1dKS4HEoB2mj7g2EsR_XKZu-gPimJHjhj' -O hfsapiens.freqfilt.tsv

In [ ]:
strict = pd.read_csv("hfsapiens.freqfilt.tsv", sep='\t')
strict.head()

In [ ]:
p_strict = strict[['POS', 'dbSNP']]

In [ ]:
str1 = p_strict['POS'].str.split(":", expand=True)

In [ ]:
str1[2] = str1[1]

In [ ]:
str1[3] = strict[['POS']]

In [ ]:
str1[0] = 'chr'+str1[0].astype(str)

In [ ]:
str1 = str1.dropna()

In [ ]:
str1[1] = str1[1].astype(int)-1
str1[2] = str1[2].astype(int)

In [ ]:
p_strict.head(1)

In [ ]:
final_strict = str1.merge(p_strict, left_on=[3], right_on='POS')

In [ ]:
final_strict.drop(['POS'], axis=1, inplace=True)
final_strict.drop([3], axis=1, inplace=True)

In [ ]:
final_strict.to_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/2021_HFstrict.bed", sep='\t', header=None, index=False)

In [ ]:
#Importing bed
hfstrict = BedTool("/home/juanandres/Desktop/TFM_raul/tmp_section2/2021_HFstrict.bed")

In [ ]:
len(hfstrict)

In [ ]:
hfstrict.head(2)

#### Intersection strict within Deserts 

In [ ]:
w_hf = akey.intersect(hfstrict, wo=True)
print(len(w_hf))

#### Intersection strict within no Deserts 

In [ ]:
wo_hf = hfstrict.intersect(akey, wo=True, v=True)
print(len(wo_hf))

#### Result | Intersection hHF with Deserts: Bed files

In [ ]:
w_hf.saveas("/home/juanandres/Desktop/TFM_raul/tmp_section2/strict_WITHIN_deserts.bed")
wo_hf.saveas("/home/juanandres/Desktop/TFM_raul/tmp_section2/strict_NOT_in_deserts.bed")

#### Intersection strict within Deserts & Pey 

In [ ]:
t = pey.intersect(akey, wo=True, f=1)
#print(len(t))
print(t.head())
t1 = hfstrict.intersect(t, wa=True, f=1)
print(t1.head())
t1.saveas("/home/juanandres/Desktop/TFM_raul/tmp_section2/strict_WITHIN_DesPey.bed")

### **1. COMPARISON | Deserts vs NO Deserts (Whole genome):** 

In [ ]:
df1 = pd.read_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/strict_WITHIN_deserts.bed", sep='\t', header=None)
df2 = pd.read_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/strict_NOT_in_deserts.bed", sep='\t', header=None)

#### Pandas dataframes 

In [ ]:
df1.drop([7], axis=1, inplace=True)
df1.columns = ['chr', 'Desert_start','Desert_end', 'chr_hg19', 'POS_start', 'POS_end', 'dbSNP']
df2.columns = ['chr', 'POS_start', 'POS_end', 'dbSNP']

In [ ]:
df1.drop(['POS_start'], axis=1, inplace=True)
df1['POS'] = df1['chr_hg19'].str.split("chr", expand=True)[1]
df1['POS'] = df1['POS'].astype(str)
df1['POS'] = df1['POS'].str.cat(df1['POS_end'].astype(str), sep=':')
df1.drop(['POS_end', 'dbSNP'], axis=1, inplace=True)

In [ ]:
df2
df2.drop(['POS_start'], axis=1, inplace=True)
df2['POS'] = df2['chr'].str.split("chr", expand=True)[1]
df2['POS'] = df2['POS'].astype(str)
df2['POS'] = df2['POS'].str.cat(df2['POS_end'].astype(str), sep=':')
df2.drop(['POS_end', 'dbSNP'], axis=1, inplace=True)
df2.head(2)

In [ ]:
df3 = pd.read_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/strict_WITHIN_DesPey.bed", sep='\t', header=None)
df3.drop([1], axis=1, inplace=True)
df3.columns = ['chr', 'SNP_pos', 'rsID']
df3['POS'] = df3['chr'].str.split("chr", expand=True)[1]
df3['POS'] = df3['POS'].astype(str)
df3['POS'] = df3['POS'].str.cat(df3['SNP_pos'].astype(str), sep=':')

In [ ]:
martin = pd.read_csv("/home/juanandres/Desktop/TFM_raul/tmp_section1/Na_high_freq.tsv", sep='\t')

In [ ]:
#Des with Martin info
martin_c1 = martin[['POS','dbSNP', 'human_DAF', 'Gene_name']]
desMartin = martin_c1.merge(df1, on = 'POS' )
desMartin.drop(['chr_hg19'], axis=1, inplace=True)
desMartin

In [ ]:
#No Deserts with Martin info
NOdesMartin = martin_c1.merge(df2, on = 'POS' )
NOdesMartin.drop(['chr'], axis=1, inplace=True)
NOdesMartin

In [ ]:
#Des+Pey with Martin info
DesPeyMartin = martin_c1.merge(df3, on = 'POS' )
DesPeyMartin.drop(['rsID'], axis=1, inplace=True)
DesPeyMartin

#### **Result | Mean frequency HF SNP in Deserts:** 

In [ ]:
desMartin['human_DAF'].mean()

#### **Result | Mean frequency HF SNP in DesertsPey:** 

In [ ]:
DesPeyMartin['human_DAF'].mean()

#### **Result | Mean frequency HF SNP in NO Deserts (whole genome):** 

In [ ]:
martin['human_DAF'].mean()

##### Result | Mean frequency HF SNP difference deserts vs NO deserts

In [ ]:
desMartin['human_DAF'].mean()-martin['human_DAF'].mean()

#### **Result | Percentage of fixed alleles in Deserts:** 

In [ ]:
len(desMartin[desMartin['human_DAF'] == 1])/len(desMartin)*100

#### **Result | Percentage of fixed alleles in NO Deserts (whole genome):** 

In [ ]:
len(NOdesMartin[NOdesMartin['human_DAF'] == 1])/len(NOdesMartin)*100

##### Result | Difference in Proportion fixed alleles Deserts vs NO Deserts (whole genome):

In [ ]:
(len(desMartin[desMartin['human_DAF'] == 1])/len(desMartin)*100)/(len(NOdesMartin[NOdesMartin['human_DAF'] == 1])/len(NOdesMartin)*100)

#### **Result | Percentage of fixed alleles in DesertsPey:** 

In [ ]:
len(DesPeyMartin[DesPeyMartin['human_DAF'] == 1])/len(DesPeyMartin)*100

In [ ]:
print(len(DesPeyMartin[DesPeyMartin['human_DAF'] == 1]))
print(len(DesPeyMartin))

In [ ]:
DesPeyMartin[DesPeyMartin['human_DAF'] == 1]['Gene_name'].unique()

### **2. COMPARISON | Deserts vs NO Deserts (Per chromosome):** 

In [ ]:
NOdesMartin['chr'] = "chr"+NOdesMartin['POS'].str.split(":", expand=True)[0]

In [ ]:
##Desert chromosomes: 
### df1_wF['chr'].unique() --> array(['chr1', 'chr3', 'chr7', 'chr8'], dtype=object)
NOdesMartin_chr = NOdesMartin[(NOdesMartin['chr'] == 'chr1') | (NOdesMartin['chr'] == 'chr3') | (NOdesMartin['chr'] == 'chr7') | (NOdesMartin['chr'] == 'chr8')]
print(len(NOdesMartin_chr))

In [ ]:
NOdesMartin_chr['chr'].unique()

#### **Result | Mean frequency HF SNP in NO Deserts (per chr):** 

In [ ]:
NOdesMartin_chr['human_DAF'].mean()

##### Result | Mean frequency HF SNP difference deserts vs NO deserts (per chr)

In [ ]:
desMartin['human_DAF'].mean()-NOdesMartin_chr['human_DAF'].mean()

#### **Result | Percentage of fixed alleles in NO Deserts (per chr):** 

In [ ]:
len(NOdesMartin_chr[NOdesMartin_chr['human_DAF'] == 1])/len(NOdesMartin_chr)*100

##### Result | Difference in Proportion fixed alleles Deserts vs NO Deserts (per chr):

In [ ]:
(len(desMartin[desMartin['human_DAF'] == 1])/len(desMartin)*100)/(len(NOdesMartin_chr[NOdesMartin_chr['human_DAF'] == 1])/len(NOdesMartin_chr)*100)

### SUMMARY TABLE

In [ ]:
summary = {'Frequency': [desMartin['human_DAF'].mean(), NOdesMartin_chr['human_DAF'].mean(), DesPeyMartin['human_DAF'].mean()], '% Fixed alleles': [len(desMartin[desMartin['human_DAF'] == 1])/len(desMartin)*100, len(NOdesMartin_chr[NOdesMartin_chr['human_DAF'] == 1])/len(NOdesMartin_chr)*100, len(DesPeyMartin[DesPeyMartin['human_DAF'] == 1])/len(DesPeyMartin)*100]}
pd.DataFrame(summary, index=['Deserts', 'No Deserts (per chr)', 'D+Pey'])

In [ ]:
summary_table = pd.DataFrame(summary, index=['Deserts', 'No Deserts (per chr)', 'D+Pey'])

In [ ]:
summary_table.to_csv("~/raul_tesina/1.data/stat_freq/Supplementary_Table_1.csv",sep = "\t")

In [ ]:
print(len(desMartin[desMartin['human_DAF'] == 1]))
print(len(desMartin))
#
print(len(NOdesMartin_chr[NOdesMartin_chr['human_DAF'] == 1]))
print(len(NOdesMartin_chr))
#
print(len(DesPeyMartin[DesPeyMartin['human_DAF'] == 1]))
print(len(DesPeyMartin))

In [ ]:
desMartin.to_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/tmp1_df1.csv", sep='\t', header=False, index=False)
NOdesMartin_chr.to_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/tmp1_df2.csv", sep='\t', header=False, index=False)
DesPeyMartin.to_csv("/home/juanandres/Desktop/TFM_raul/tmp_section2/tmp1_df3.csv", sep='\t', header=False, index=False)